# Installation Requirements

In [ ]:
!pip uninstall --quiet tensorflow --yes
!pip uninstall --quiet kapre --yes
!pip install --quiet tensorflow==1.14.0
!pip install --quiet stable_baselines
!pip uninstall --quiet optuna --yes
!pip install --quiet optuna==2.10.0
!pip uninstall --quiet tensorboard-plugin-wit --yes
!pip install --quiet "gym==0.19.0"

     |████████████████████████████████| 109.3 MB 49 kB/s 
     |████████████████████████████████| 488 kB 55.9 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 3.1 MB 37.1 MB/s 


In [ ]:
# Set working directory
%cd /content/drive/MyDrive/ColabNotebooks/Masterarbeit/RLModel/src

# run config.py to load all required settings  
%run config.py

# Analyze Hypertuning

In [ ]:
import optuna
study_name = 'hyperstudy_f1_new'
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.load_study(study_name=study_name, storage=storage_name)
study.trials_dataframe(attrs=("number", "value", "params", "state"))

,number,value,params_batch_size,params_buffer_size,params_double_q,params_dueling,params_exploration_final_eps,params_exploration_fraction,params_gamma,params_layer_1,...,params_negative_reward_good_wafer,params_param_lambda,params_param_noise,params_positive_reward_bad_wafer,params_positive_reward_good_wafer,params_prioritized_replay,params_prioritized_replay_alpha,params_prioritized_replay_beta0,params_target_network_update_freq,state
0,0,0.500608,32,30000,True,False,0.1,0.0100,0.900,32,...,1.0,0.0001,False,0.3,1.0,True,0.60,0.5,1500,COMPLETE
1,1,0.634427,128,50000,True,False,0.3,0.0001,0.900,32,...,0.8,0.0010,True,0.8,0.5,False,NaN,NaN,500,COMPLETE
2,2,0.659156,128,20000,False,False,0.1,0.0010,0.900,64,...,1.0,0.0010,False,0.8,1.0,True,0.70,0.4,1000,COMPLETE
3,3,0.644503,128,5000,True,True,0.1,0.0001,0.900,32,...,0.8,0.0100,False,0.5,0.3,False,NaN,NaN,1500,COMPLETE
4,4,0.433829,32,40000,False,False,0.2,0.0010,0.999,128,...,0.8,0.0001,False,0.5,0.5,True,0.60,0.6,500,COMPLETE
5,5,0.537755,64,5000,False,False,0.1,0.0010,0.950,64,...,0.5,0.0100,True,0.3,0.8,False,NaN,NaN,1500,COMPLETE
6,6,0.587907,128,5000,False,False,0.1,0.0100,0.999,128,...,0.8,0.0010,False,0.3,0.8,False,NaN,NaN,1500,COMPLETE
7,7,0.629033,64,40000,False,True,0.2,0.0001,0.900,64,...,0.8,0.0010,True,0.5,0.3,False,NaN,NaN,500,COMPLETE
8,8,0.512370,32,5000,False,False,0.3,0.0001,0.999,64,...,1.0,0.0001,True,0.5,0.5,False,NaN,NaN,1500,COMPLETE
9,9,0.635573,128,10000,False,False,0.3,0.0010,0.950,128,...,0.8,0.0100,False,0.3,0.5,True,0.65,0.6,1500,COMPLETE


In [ ]:
from optuna.visualization import plot_param_importances
plot_param_importances(study)

## Conclusion
trial 36 reaches the best f1 score
- 1.Fold: 35.000 training steps
- 2.Fold: 50.000 training steps
- 3.Fold: 45.000 training steps
- 4.Fold: 35.000 training steps

--> Training on all training data with a training steps length of mean(35,50,45,35) = 41.250

# Train best model

In [ ]:
!python -W ignore train.py --path '../models/best_models/best_model_f1_new/' --study_name 'hyperstudy_f1_new' --timesteps 41250 --custom_callback --train_best_hypermodel


---------------------------------------------------------------------------------------
Model Evaluation on TRAINING DATA (last 100 episodes)
---------------------------------------------------------------------------------------
[1.21%/100%] train_steps 500 (eps 179), mean_reward: -0.16 +/- 0.67, acc: 0.49, mean_timesteps: 2.93 +/- 1.90
[2.42%/100%] train_steps 1000 (eps 353), mean_reward: -0.12 +/- 0.65, acc: 0.53, mean_timesteps: 2.86 +/- 1.46
[3.64%/100%] train_steps 1500 (eps 523), mean_reward: -0.13 +/- 0.65, acc: 0.53, mean_timesteps: 3.01 +/- 1.60
[4.85%/100%] train_steps 2000 (eps 699), mean_reward: -0.33 +/- 0.62, acc: 0.38, mean_timesteps: 2.89 +/- 1.76
[6.06%/100%] train_steps 2500 (eps 869), mean_reward: -0.25 +/- 0.61, acc: 0.43, mean_timesteps: 2.99 +/- 1.48
[7.27%/100%] train_steps 3000 (eps 1047), mean_reward: -0.24 +/- 0.66, acc: 0.43, mean_timesteps: 2.79 +/- 1.56
[8.48%/100%] train_steps 3500 (eps 1496), mean_reward: 0.10 +/- 0.45, acc: 0.72, mean_timesteps: 1.15 +

In [ ]:
!python -W ignore evaluate.py --path '../models/best_models/best_model_f1_new/' --study_name 'hyperstudy_f1_new' --train_best_hypermodel --evaluation_summary

Evaluation Summary on Test_Data ( 292 data points)
                model  mean_reward       acc  sensitivity  precision        f1  specificity       gm  mean_timesteps  mean_timesteps_good  mean_timesteps_bad best_mean_reward best_f1 best_sensitivity best_specificity best_acc best_gm
0  rl_model_final.zip     0.271943  0.873288     0.784314   0.606061  0.683761     0.892116  0.83648        1.263699             1.097345            1.833333                x       x                x                x        x       x

Evaluation Summary on Train_Data ( 679 data points)
                model  mean_reward       acc  sensitivity  precision   f1  specificity      gm  mean_timesteps  mean_timesteps_good  mean_timesteps_bad best_mean_reward best_f1 best_sensitivity best_specificity best_acc best_gm
0  rl_model_final.zip     0.335186  0.929308     0.806723   0.793388  0.8     0.955357  0.8779        1.238586             1.159498            1.603306                x       x                x       